In [28]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [8]:
df = pd.read_csv(r"\Users\Admin\Desktop\2025-ZOOM\Docker-IaC\yellow_tripdata_2024-01.csv")
df.head(10)


,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.00
1,1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.00
2,2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.00
3,3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.00
4,4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.00
5,5,1,2024-01-01 00:54:08,2024-01-01 01:26:31,1.0,4.70,1.0,N,148,141,1,29.6,3.5,0.5,6.90,0.0,1.0,41.50,2.5,0.00
6,6,2,2024-01-01 00:49:44,2024-01-01 01:15:47,2.0,10.82,1.0,N,138,181,1,45.7,6.0,0.5,10.00,0.0,1.0,64.95,0.0,1.75
7,7,1,2024-01-01 00:30:40,2024-01-01 00:58:40,0.0,3.00,1.0,N,246,231,2,25.4,3.5,0.5,0.00,0.0,1.0,30.40,2.5,0.00
8,8,2,2024-01-01 00:26:01,2024-01-01 00:54:12,1.0,5.44,1.0,N,161,261,2,31.0,1.0,0.5,0.00,0.0,1.0,36.00,2.5,0.00
9,9,2,2024-01-01 00:28:08,2024-01-01 00:29:16,1.0,0.04,1.0,N,113,113,2,3.0,1.0,0.5,0.00,0.0,1.0,8.00,2.5,0.00


In [10]:
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime )
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

In [11]:
print(pd.io.sql.get_schema(df,name="yellow_taxi"))

CREATE TABLE "yellow_taxi" (
"Unnamed: 0" INTEGER,
  "VendorID" INTEGER,
  "tpep_pickup_datetime" TIMESTAMP,
  "tpep_dropoff_datetime" TIMESTAMP,
  "passenger_count" REAL,
  "trip_distance" REAL,
  "RatecodeID" REAL,
  "store_and_fwd_flag" TEXT,
  "PULocationID" INTEGER,
  "DOLocationID" INTEGER,
  "payment_type" INTEGER,
  "fare_amount" REAL,
  "extra" REAL,
  "mta_tax" REAL,
  "tip_amount" REAL,
  "tolls_amount" REAL,
  "improvement_surcharge" REAL,
  "total_amount" REAL,
  "congestion_surcharge" REAL,
  "Airport_fee" REAL
)


In [29]:
conn = psycopg2.connect(
    host='localhost',
    port=5433,
    user='postgres',
    database='postgres',
    password='mGFeanZG'
)

cursor = conn.cursor()

In [30]:
engine = create_engine('postgresql+psycopg2://', creator=lambda:conn)

In [31]:
create_query_file = open("../data/create_table.sql")
create_query = create_query_file.read()
create_query

'CREATE TABLE "yellow_taxi" (\n"Unnamed: 0" INTEGER,\n  "VendorID" INTEGER,\n  "tpep_pickup_datetime" TIMESTAMP,\n  "tpep_dropoff_datetime" TIMESTAMP,\n  "passenger_count" REAL,\n  "trip_distance" REAL,\n  "RatecodeID" REAL,\n  "store_and_fwd_flag" TEXT,\n  "PULocationID" INTEGER,\n  "DOLocationID" INTEGER,\n  "payment_type" INTEGER,\n  "fare_amount" REAL,\n  "extra" REAL,\n  "mta_tax" REAL,\n  "tip_amount" REAL,\n  "tolls_amount" REAL,\n  "improvement_surcharge" REAL,\n  "total_amount" REAL,\n  "congestion_surcharge" REAL,\n  "Airport_fee" REAL\n)'

In [32]:
cursor.execute(create_query)

In [37]:
df_iter = pd.read_csv(r"\Users\Admin\Desktop\2025-ZOOM\Docker-IaC\yellow_tripdata_2024-01.csv",iterator=True,chunksize=1000)
chunk = next(df_iter)
merge_query_file = open("../data/insert.sql")
merge_query = merge_query_file.read()
merge_query

'INSERT INTO "yellow_taxi" \n("Unnamed: 0", "VendorID","tpep_pickup_datetime","tpep_dropoff_datetime","passenger_count","trip_distance","RatecodeID","store_and_fwd_flag","PULocationID","DOLocationID","payment_type","fare_amount","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","congestion_surcharge","Airport_fee")\nVALUES(\'%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s\')'

In [39]:
for chunk in df_iter:
    cursor.executemany(merge_query,chunk.to_dict(orient="records"))

TypeError: dict is not a sequence

In [24]:
df.head(n=0)

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
